## Second Part: Write queries that directly answer predetermined questions from a business stakeholder
I am using SQLite database through SQL Alchemy python to implement the SQL database for the queries.

In [120]:
import pandas as pd
import json
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, Float, ForeignKey, text
from datetime import datetime, timezone

In [122]:
# Load JSON files into pandas DataFrames
users_df = pd.read_json('/Users/spartan/Downloads/users.json', lines=True)
receipts_df = pd.read_json('/Users/spartan/Downloads/receipts.json', lines=True)
brands_df = pd.read_json('/Users/spartan/Downloads/brands.json', lines=True)

#### Step 1 - Clean and Normalize Data


In [125]:
# Clean '_id' columns by extracting '$oid'
users_df['_id'] = users_df['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else x)
receipts_df['_id'] = receipts_df['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else x)
brands_df['_id'] = brands_df['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else x)

# Convert date fields from UNIX epoch format to datetime (timezone-aware)
users_df['createdDate'] = users_df['createdDate'].apply(lambda x: datetime.fromtimestamp(x['$date'] / 1000, tz=timezone.utc) if isinstance(x, dict) else x)
users_df['lastLogin'] = users_df['lastLogin'].apply(lambda x: datetime.fromtimestamp(x['$date'] / 1000, tz=timezone.utc) if isinstance(x, dict) else x)
receipts_df['createDate'] = receipts_df['createDate'].apply(lambda x: datetime.fromtimestamp(x['$date'] / 1000, tz=timezone.utc) if isinstance(x, dict) else x)

In [127]:
# Flatten the 'rewardsReceiptItemList' by converting it to JSON strings
receipts_df['rewardsReceiptItemList'] = receipts_df['rewardsReceiptItemList'].apply(lambda x: json.dumps(x) if isinstance(x, list) else x)

In [129]:
# Handle other date fields in receipts_df
date_fields = ['dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for field in date_fields:
    receipts_df[field] = receipts_df[field].apply(lambda x: datetime.fromtimestamp(x['$date'] / 1000, tz=timezone.utc) if isinstance(x, dict) else x)

In [131]:
#Serialize the 'cpg' column to a JSON string
brands_df['cpg'] = brands_df['cpg'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

In [133]:
# Renaming _id to respective column names
users_df.rename(columns={'_id': 'user_id'}, inplace=True)
receipts_df.rename(columns={'_id': 'receipt_id'}, inplace=True)
brands_df.rename(columns={'_id': 'brand_id'}, inplace=True)

#### Step 2 - Creating an SQLAlchemy Engine

In [139]:
# Create an in-memory SQLite database
engine = create_engine('sqlite:///my_database.db', echo=True)

In [141]:
# Define metadata to keep track of the table schemas
metadata = MetaData()

### Step 3 Defining Tables

In [143]:
# Define the users table
users_table = Table(
    'users', metadata,
    Column('user_id', String, primary_key=True),
    Column('active', Integer),
    Column('createdDate', String),
    Column('lastLogin', String),
    Column('role', String),
    Column('state', String),
    Column('signUpSource', String)
)

# Define the receipts table
receipts_table = Table(
    'receipts', metadata,
    Column('receipt_id', String, primary_key=True),
    Column('userId', String, ForeignKey('users.user_id')),
    Column('pointsEarned', Float),
    Column('bonusPointsEarned', Float),
    Column('bonusPointsEarnedReason', String),
    Column('createDate', String),
    Column('dateScanned', String),
    Column('finishedDate', String),
    Column('modifyDate', String),
    Column('pointsAwardedDate', String),
    Column('purchaseDate', String),
    Column('purchasedItemCount', Float),
    Column('rewardsReceiptItemList', String),
    Column('rewardsReceiptStatus', String),
    Column('totalSpent', Float)
)

# Define the brands table
brands_table = Table(
    'brands', metadata,
    Column('brand_id', String, primary_key=True),
    Column('barcode', String),
    Column('category', String),
    Column('categoryCode', String),
    Column('cpg', String),
    Column('name', String),
    Column('topBrand', Float),
    Column('brandCode', String)
)

# Define the items table
items_table = Table(
    'items', metadata,
    Column('itemId', Integer, primary_key=True, autoincrement=True),
    Column('receiptId', String, ForeignKey('receipts.receipt_id'), nullable=False),
    Column('barcode', String, ForeignKey('brands.barcode'), nullable=True),
    Column('description', String),
    Column('itemPrice', Float),
    Column('finalPrice', Float),
    Column('quantityPurchased', Integer)
)

# Create all tables in the SQLite database
metadata.create_all(engine)

2024-10-30 00:24:03,599 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 00:24:03,601 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-10-30 00:24:03,602 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:24:03,604 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("receipts")
2024-10-30 00:24:03,605 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:24:03,607 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("brands")
2024-10-30 00:24:03,609 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:24:03,611 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("items")
2024-10-30 00:24:03,612 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:24:03,614 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("items")
2024-10-30 00:24:03,615 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:24:03,621 INFO sqlalchemy.engine.Engine 
CREATE TABLE items (
	"itemId" INTEGER NOT NULL, 
	"receiptId" VARCHAR NOT NULL, 
	barcode VARCHAR, 
	des

#### Step 4 - Insert DataFrames into SQL Tables

In [146]:
# Insert users into the SQL table
users_df.to_sql('users', con=engine, if_exists='replace', index=False)

# Insert receipts into the SQL table
receipts_df.to_sql('receipts', con=engine, if_exists='replace', index=False)

# Insert brands into the SQL table
brands_df.to_sql('brands', con=engine, if_exists='replace', index=False)


2024-10-30 00:25:37,851 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 00:25:37,856 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-10-30 00:25:37,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:25:37,858 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-10-30 00:25:37,858 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:25:37,859 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-10-30 00:25:37,859 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:25:37,860 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-10-30 00:25:37,861 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:25:37,861 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("users")
2024-10-30 00:25:37,861 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 00:25:3

1167

### Extracting item details and inserting in item table

In [208]:
#  Parse the 'rewardsReceiptItemList' JSON strings and extract item details
items_data = []

for _, row in receipts_df.iterrows():
    receipt_id = row['receipt_id']
    rewards_items = row['rewardsReceiptItemList']
    if pd.notna(rewards_items):
        items_list = json.loads(rewards_items)
        for item in items_list:
            # Extract item details if they exist in the JSON
            barcode = item.get('barcode')
            description = item.get('description')
            item_price = item.get('itemPrice')
            final_price = item.get('finalPrice')
            quantity_purchased = item.get('quantityPurchased')
            
            # Append the extracted item details to the list
            items_data.append({
                'receiptId': receipt_id,
                'barcode': barcode,
                'description': description,
                'itemPrice': item_price,
                'finalPrice': final_price,
                'quantityPurchased': quantity_purchased
            })

# Step 2: Create a DataFrame for the 'items' table
items_df = pd.DataFrame(items_data)

# Step 3: Insert the items DataFrame into the 'items' SQL table
items_df.to_sql('items', con=engine, if_exists='replace', index=False)

2024-10-30 01:23:45,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 01:23:45,229 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("items")
2024-10-30 01:23:45,230 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 01:23:45,231 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("items")
2024-10-30 01:23:45,231 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 01:23:45,232 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-10-30 01:23:45,232 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 01:23:45,235 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-10-30 01:23:45,235 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 01:23:45,239 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("items")
2024-10-30 01:23:45,239 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-30 01:23:4

6941

### Step 5 Running the Queries

In [215]:
from sqlalchemy import func, text
from datetime import datetime, timedelta

# Calculate date ranges for recent and previous month
now = datetime.now()
start_of_this_month = now.replace(day=1)
start_of_last_month = (start_of_this_month - timedelta(days=1)).replace(day=1)
end_of_last_month = start_of_this_month - timedelta(days=1)

#### 1. What are the top 5 brands by receipts scanned for most recent month?

In [226]:
with engine.connect() as conn:
    # 1. Top 5 brands by receipts scanned for the most recent month
    top_5_brands_query = text("""
        SELECT b.name, COUNT(r.receipt_id) AS receipt_count
        FROM receipts r
        JOIN items i ON r.receipt_id = i.receiptId
        JOIN brands b ON i.barcode = b.barcode
        WHERE r.createDate >= date('now', '-1 month')
        GROUP BY b.name
        ORDER BY receipt_count DESC
        LIMIT 5;
    """)

    try:
        top_5_brands_result = conn.execute(top_5_brands_query).fetchall()
        print("\nTop 5 Brands by Receipts Scanned for Most Recent Month:")
        for row in top_5_brands_result:
            print(row)
    except Exception as e:
        print(f"An error occurred while fetching top 5 brands: {e}")

2024-10-30 01:35:06,316 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 01:35:06,319 INFO sqlalchemy.engine.Engine 
        SELECT b.name, COUNT(r.receipt_id) AS receipt_count
        FROM receipts r
        JOIN items i ON r.receipt_id = i.receiptId
        JOIN brands b ON i.barcode = b.barcode
        WHERE r.createDate >= date('now', '-1 month')
        GROUP BY b.name
        ORDER BY receipt_count DESC
        LIMIT 5;
    
2024-10-30 01:35:06,320 INFO sqlalchemy.engine.Engine [generated in 0.00407s] ()

Top 5 Brands by Receipts Scanned for Most Recent Month:
2024-10-30 01:35:06,324 INFO sqlalchemy.engine.Engine ROLLBACK


#### 2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [229]:
with engine.connect() as conn:
    # 2. Ranking comparison of top 5 brands by receipts scanned for recent and previous month
    recent_month_query = text("""
        SELECT b.name, COUNT(r.receipt_id) AS receipt_count
        FROM receipts r
        JOIN items i ON r.receipt_id = i.receiptId
        JOIN brands b ON i.barcode = b.barcode
        WHERE r.createDate BETWEEN :start_of_this_month AND :now
        GROUP BY b.name
        ORDER BY receipt_count DESC
        LIMIT 5;
    """)

    last_month_query = text("""
        SELECT b.name, COUNT(r.receipt_id) AS receipt_count
        FROM receipts r
        JOIN items i ON r.receipt_id = i.receiptId
        JOIN brands b ON i.barcode = b.barcode
        WHERE r.createDate BETWEEN :start_of_last_month AND :end_of_last_month
        GROUP BY b.name
        ORDER BY receipt_count DESC
        LIMIT 5;
    """)

    # Execute queries for recent and previous month
    recent_month_result = conn.execute(recent_month_query, {
        'start_of_this_month': start_of_this_month, 
        'now': now
    }).fetchall()
    last_month_result = conn.execute(last_month_query, {
        'start_of_last_month': start_of_last_month, 
        'end_of_last_month': end_of_last_month
    }).fetchall()

    print("Top 5 Brands for Recent Month:")
    for row in recent_month_result:
        print(row)

    print("\nTop 5 Brands for Previous Month:")
    for row in last_month_result:
        print(row)

2024-10-30 01:35:20,800 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 01:35:20,801 INFO sqlalchemy.engine.Engine 
        SELECT b.name, COUNT(r.receipt_id) AS receipt_count
        FROM receipts r
        JOIN items i ON r.receipt_id = i.receiptId
        JOIN brands b ON i.barcode = b.barcode
        WHERE r.createDate BETWEEN ? AND ?
        GROUP BY b.name
        ORDER BY receipt_count DESC
        LIMIT 5;
    
2024-10-30 01:35:20,801 INFO sqlalchemy.engine.Engine [cached since 2636s ago] (datetime.datetime(2024, 10, 1, 1, 27, 30, 585509), datetime.datetime(2024, 10, 30, 1, 27, 30, 585509))
2024-10-30 01:35:20,803 INFO sqlalchemy.engine.Engine 
        SELECT b.name, COUNT(r.receipt_id) AS receipt_count
        FROM receipts r
        JOIN items i ON r.receipt_id = i.receiptId
        JOIN brands b ON i.barcode = b.barcode
        WHERE r.createDate BETWEEN ? AND ?
        GROUP BY b.name
        ORDER BY receipt_count DESC
        LIMIT 5;
    
2024-10-30 01:35:20,80

#### 3. Which brand has the most transactions among users who were created within the past 6 months?

In [221]:
with engine.connect() as conn:
    most_transactions_query = text("""
        SELECT b.name, COUNT(r.receipt_id) AS transaction_count
        FROM users u
        JOIN receipts r ON u.user_id = r.userId
        JOIN items i ON r.receipt_id = i.receiptId
        JOIN brands b ON i.barcode = b.barcode
        WHERE u.createdDate >= :six_months_ago
        GROUP BY b.name
        ORDER BY transaction_count DESC
        LIMIT 1;
    """)

    most_transactions_result = conn.execute(most_transactions_query, {'six_months_ago': six_months_ago}).fetchone()
    print("\nBrand with Most Transactions Among Users Created in Last 6 Months:")
    print(most_transactions_result)

2024-10-30 01:30:29,803 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 01:30:29,804 INFO sqlalchemy.engine.Engine 
        SELECT b.name, COUNT(r.receipt_id) AS transaction_count
        FROM users u
        JOIN receipts r ON u.user_id = r.userId
        JOIN items i ON r.receipt_id = i.receiptId
        JOIN brands b ON i.barcode = b.barcode
        WHERE u.createdDate >= ?
        GROUP BY b.name
        ORDER BY transaction_count DESC
        LIMIT 1;
    
2024-10-30 01:30:29,805 INFO sqlalchemy.engine.Engine [cached since 1780s ago] (datetime.datetime(2024, 5, 3, 1, 0, 49, 411960),)

Brand with Most Transactions Among Users Created in Last 6 Months:
None
2024-10-30 01:30:29,807 INFO sqlalchemy.engine.Engine ROLLBACK


#### There 0 brands so let's check if there are any users created in last 6 months

In [256]:
with engine.connect() as conn:
    users_query = text("""
        SELECT COUNT(*)
        FROM users
        WHERE createdDate >= :six_months_ago
    """)
    users_result = conn.execute(users_query, {'six_months_ago': six_months_ago}).scalar()
    print(f"Number of users created in the last 6 months: {users_result}")


2024-10-30 02:01:05,379 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 02:01:05,381 INFO sqlalchemy.engine.Engine 
        SELECT COUNT(*)
        FROM users
        WHERE createdDate >= ?
    
2024-10-30 02:01:05,382 INFO sqlalchemy.engine.Engine [cached since 3536s ago] (datetime.datetime(2024, 5, 3, 1, 0, 49, 411960),)
Number of users created in the last 6 months: 0
2024-10-30 02:01:05,385 INFO sqlalchemy.engine.Engine ROLLBACK


#### Checking the Dates

In [202]:
from sqlalchemy import text

with engine.connect() as conn:
    date_columns_query = text("""
        SELECT createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, purchaseDate
        FROM receipts
        LIMIT 10;
    """)

    try:
        date_columns_result = conn.execute(date_columns_query).fetchall()
        if date_columns_result:
            print("\nSample Dates from Receipts Table:")
            for row in date_columns_result:
                print(row)
        else:
            print("\nNo records found in Receipts Table.")
    except Exception as e:
        print(f"An error occurred: {e}")


2024-10-30 01:11:05,184 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 01:11:05,187 INFO sqlalchemy.engine.Engine 
        SELECT createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, purchaseDate
        FROM receipts
        LIMIT 10;
    
2024-10-30 01:11:05,188 INFO sqlalchemy.engine.Engine [cached since 184.4s ago] ()

Sample Dates from Receipts Table:
('2021-01-03 15:25:31.000000', '2021-01-03 15:25:31.000000', '2021-01-03 15:25:31.000000', '2021-01-03 15:25:36.000000', '2021-01-03 15:25:31.000000', '2021-01-03 00:00:00.000000')
('2021-01-03 15:24:43.000000', '2021-01-03 15:24:43.000000', '2021-01-03 15:24:43.000000', '2021-01-03 15:24:48.000000', '2021-01-03 15:24:43.000000', '2021-01-02 15:24:43.000000')
('2021-01-03 15:25:37.000000', '2021-01-03 15:25:37.000000', None, '2021-01-03 15:25:42.000000', None, '2021-01-03 00:00:00.000000')
('2021-01-03 15:25:34.000000', '2021-01-03 15:25:34.000000', '2021-01-03 15:25:34.000000', '2021-01-03 15:25:39.000000

## All the sample dates in the receipts table are from the year 2021. This is why queries that depend on recent data (such as transactions in the last 6 months) are returning None or empty results. The data in this database does not contain any recent records beyond 2021.

### Adjusting date ranges for the existing data in the database

In [267]:
# Adjusting date ranges for the existing data in the database (January 2021 and December 2020)
start_of_this_month = datetime(2021, 1, 1)
end_of_this_month = datetime(2021, 1, 31)

start_of_last_month = datetime(2020, 12, 1)
end_of_last_month = datetime(2020, 12, 31)

with engine.connect() as conn:
    date_columns_query = text("""
        SELECT createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, purchaseDate
        FROM receipts
        LIMIT 10;
    """)

    try:
        date_columns_result = conn.execute(date_columns_query).fetchall()
        if date_columns_result:
            print("\nSample Dates from Receipts Table:")
            for row in date_columns_result:
                print(row)
        else:
            print("\nNo records found in Receipts Table.")
    except Exception as e:
        print(f"An error occurred: {e}")

2024-10-30 02:06:25,111 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 02:06:25,112 INFO sqlalchemy.engine.Engine 
        SELECT createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, purchaseDate
        FROM receipts
        LIMIT 10;
    
2024-10-30 02:06:25,113 INFO sqlalchemy.engine.Engine [cached since 3504s ago] ()

Sample Dates from Receipts Table:
('2021-01-03 15:25:31.000000', '2021-01-03 15:25:31.000000', '2021-01-03 15:25:31.000000', '2021-01-03 15:25:36.000000', '2021-01-03 15:25:31.000000', '2021-01-03 00:00:00.000000')
('2021-01-03 15:24:43.000000', '2021-01-03 15:24:43.000000', '2021-01-03 15:24:43.000000', '2021-01-03 15:24:48.000000', '2021-01-03 15:24:43.000000', '2021-01-02 15:24:43.000000')
('2021-01-03 15:25:37.000000', '2021-01-03 15:25:37.000000', None, '2021-01-03 15:25:42.000000', None, '2021-01-03 00:00:00.000000')
('2021-01-03 15:25:34.000000', '2021-01-03 15:25:34.000000', '2021-01-03 15:25:34.000000', '2021-01-03 15:25:39.000000'

In [269]:
# Queries for top 5 brands for January 2021 and December 2020
recent_month_query = text("""
    SELECT b.name, COUNT(r.receipt_id) AS receipt_count
    FROM receipts r
    JOIN items i ON r.receipt_id = i.receiptId
    JOIN brands b ON i.barcode = b.barcode
    WHERE r.createDate BETWEEN :start_of_this_month AND :end_of_this_month
    GROUP BY b.name
    ORDER BY receipt_count DESC
    LIMIT 5;
""")

last_month_query = text("""
    SELECT b.name, COUNT(r.receipt_id) AS receipt_count
    FROM receipts r
    JOIN items i ON r.receipt_id = i.receiptId
    JOIN brands b ON i.barcode = b.barcode
    WHERE r.createDate BETWEEN :start_of_last_month AND :end_of_last_month
    GROUP BY b.name
    ORDER BY receipt_count DESC
    LIMIT 5;
""")

with engine.connect() as conn:
    try:
        # Execute queries for recent and previous month
        recent_month_result = conn.execute(recent_month_query, {
            'start_of_this_month': start_of_this_month,
            'end_of_this_month': end_of_this_month
        }).fetchall()

        last_month_result = conn.execute(last_month_query, {
            'start_of_last_month': start_of_last_month,
            'end_of_last_month': end_of_last_month
        }).fetchall()

        print("Top 5 Brands for January 2021:")
        for row in recent_month_result:
            print(row)

        print("\nTop 5 Brands for December 2020:")
        for row in last_month_result:
            print(row)

    except Exception as e:
        print(f"An error occurred: {e}")


2024-10-30 02:08:03,973 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 02:08:03,986 INFO sqlalchemy.engine.Engine 
    SELECT b.name, COUNT(r.receipt_id) AS receipt_count
    FROM receipts r
    JOIN items i ON r.receipt_id = i.receiptId
    JOIN brands b ON i.barcode = b.barcode
    WHERE r.createDate BETWEEN ? AND ?
    GROUP BY b.name
    ORDER BY receipt_count DESC
    LIMIT 5;

2024-10-30 02:08:03,987 INFO sqlalchemy.engine.Engine [generated in 0.01429s] (datetime.datetime(2021, 1, 1, 0, 0), datetime.datetime(2021, 1, 31, 0, 0))
2024-10-30 02:08:04,005 INFO sqlalchemy.engine.Engine 
    SELECT b.name, COUNT(r.receipt_id) AS receipt_count
    FROM receipts r
    JOIN items i ON r.receipt_id = i.receiptId
    JOIN brands b ON i.barcode = b.barcode
    WHERE r.createDate BETWEEN ? AND ?
    GROUP BY b.name
    ORDER BY receipt_count DESC
    LIMIT 5;

2024-10-30 02:08:04,006 INFO sqlalchemy.engine.Engine [generated in 0.00142s] (datetime.datetime(2020, 12, 1, 0, 0), dateti

#### 4. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [236]:
with engine.connect() as conn:
    avg_spend_query = text("""
        SELECT rewardsReceiptStatus, AVG(totalSpent) AS average_spend
        FROM receipts
        WHERE rewardsReceiptStatus IN ('Accepted', 'Rejected')
        GROUP BY rewardsReceiptStatus;
    """)

    try:
        avg_spend_result = conn.execute(avg_spend_query).fetchall()
        print("\nAverage Spend from Receipts with 'Accepted' or 'Rejected' Status:")
        for row in avg_spend_result:
            print(row)
    except Exception as e:
        print(f"An error occurred while fetching average spend: {e}")

2024-10-30 01:38:54,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 01:38:54,482 INFO sqlalchemy.engine.Engine 
        SELECT rewardsReceiptStatus, AVG(totalSpent) AS average_spend
        FROM receipts
        WHERE rewardsReceiptStatus IN ('Accepted', 'Rejected')
        GROUP BY rewardsReceiptStatus;
    
2024-10-30 01:38:54,483 INFO sqlalchemy.engine.Engine [generated in 0.00343s] ()

Average Spend from Receipts with 'Accepted' or 'Rejected' Status:
2024-10-30 01:38:54,515 INFO sqlalchemy.engine.Engine ROLLBACK


#### Since it is returning empty let's check if there are any records with rewardsReceiptStatus as 'Accepted' or 'Rejected'.

In [238]:
with engine.connect() as conn:
    status_check_query = text("""
        SELECT rewardsReceiptStatus, COUNT(*)
        FROM receipts
        WHERE rewardsReceiptStatus IN ('Accepted', 'Rejected')
        GROUP BY rewardsReceiptStatus;
    """)

    try:
        status_check_result = conn.execute(status_check_query).fetchall()
        if status_check_result:
            print("\nCount of Receipts with 'Accepted' or 'Rejected' Status:")
            for row in status_check_result:
                print(row)
        else:
            print("\nNo receipts found with 'Accepted' or 'Rejected' status.")
    except Exception as e:
        print(f"An error occurred while checking receipt status: {e}")


2024-10-30 01:40:11,358 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 01:40:11,360 INFO sqlalchemy.engine.Engine 
        SELECT rewardsReceiptStatus, COUNT(*)
        FROM receipts
        WHERE rewardsReceiptStatus IN ('Accepted', 'Rejected')
        GROUP BY rewardsReceiptStatus;
    
2024-10-30 01:40:11,362 INFO sqlalchemy.engine.Engine [generated in 0.00360s] ()

No receipts found with 'Accepted' or 'Rejected' status.
2024-10-30 01:40:11,370 INFO sqlalchemy.engine.Engine ROLLBACK


#### Since no receipts found let's check the values existing in the rewardsReceiptStatus column.

In [251]:
with engine.connect() as conn:
    unique_status_query = text("""
        SELECT DISTINCT rewardsReceiptStatus
        FROM receipts;
    """)

    try:
        unique_status_result = conn.execute(unique_status_query).fetchall()
        if unique_status_result:
            print("\nUnique Rewards Receipt Status Values:")
            for row in unique_status_result:
                print(row)
        else:
            print("\nNo rewardsReceiptStatus found in Receipts Table.")
    except Exception as e:
        print(f"An error occurred while fetching unique receipt status values: {e}")


2024-10-30 01:49:45,139 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 01:49:45,141 INFO sqlalchemy.engine.Engine 
        SELECT DISTINCT rewardsReceiptStatus
        FROM receipts;
    
2024-10-30 01:49:45,142 INFO sqlalchemy.engine.Engine [cached since 520.4s ago] ()

Unique Rewards Receipt Status Values:
('FINISHED',)
('REJECTED',)
('FLAGGED',)
('SUBMITTED',)
('PENDING',)
2024-10-30 01:49:45,151 INFO sqlalchemy.engine.Engine ROLLBACK


#### Accepted is not present, we can use the appropriate available status. FINISHED and REJECTED

In [242]:
with engine.connect() as conn:
    avg_spend_query = text("""
        SELECT rewardsReceiptStatus, AVG(totalSpent) AS average_spend
        FROM receipts
        WHERE rewardsReceiptStatus IN ('FINISHED', 'REJECTED')
        GROUP BY rewardsReceiptStatus;
    """)

    try:
        avg_spend_result = conn.execute(avg_spend_query).fetchall()
        if avg_spend_result:
            print("\nAverage Spend from Receipts with 'FINISHED' or 'REJECTED' Status:")
            for row in avg_spend_result:
                print(row)
        else:
            print("\nNo receipts found with 'FINISHED' or 'REJECTED' status.")
    except Exception as e:
        print(f"An error occurred while fetching average spend: {e}")


2024-10-30 01:42:07,678 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-30 01:42:07,681 INFO sqlalchemy.engine.Engine 
        SELECT rewardsReceiptStatus, AVG(totalSpent) AS average_spend
        FROM receipts
        WHERE rewardsReceiptStatus IN ('FINISHED', 'REJECTED')
        GROUP BY rewardsReceiptStatus;
    
2024-10-30 01:42:07,681 INFO sqlalchemy.engine.Engine [generated in 0.00433s] ()

Average Spend from Receipts with 'FINISHED' or 'REJECTED' Status:
('FINISHED', 80.85430501930502)
('REJECTED', 23.32605633802817)
2024-10-30 01:42:07,691 INFO sqlalchemy.engine.Engine ROLLBACK


### Based on the results:
#### 1. Average Spend for 'FINISHED' Receipts: 80.85
#### 2. Average Spend for 'REJECTED' Receipts: 23.33
#### The average spend is greater for receipts with a status of 'FINISHED' compared to 'REJECTED'.